# **Project Alpha**

## Dependencies

In [1]:
import json
import pandas as pd
import numpy as np
import os



## Settings


In [2]:
# Set the display option
pd.set_option('display.expand_frame_repr', False)

data_file = 'data/Sample data.json'
clean_file = 'data/clean_data.json'

## Test

In [3]:

# Check if the file exists
if os.path.exists(data_file):
    # Try to load and parse the JSON in the file
    try:
        with open(data_file, 'r') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        print(df.head())
    except Exception as e:
        print(f"An error occurred: {type(e).__name__}, {str(e)}")
else:
    print(f"The file {data_file} does not exist.")

An error occurred: JSONDecodeError, Expecting property name enclosed in double quotes: line 3 column 4 (char 8)


In [4]:
# Check if the file exists
if os.path.exists(data_file):
    # Read the file
    with open(data_file, 'r') as f:
        data = f.read()

    # Replace single quotes with double quotes
    data = data.replace("'", '"')

    # Write the result back to the file
    with open(clean_file, 'w') as f:
        f.write(data)
    print(f"Cleaned data written to {clean_file}.")
else:
    print(f"The file {clean_file} does not exist.")

Cleaned data written to data/clean_data.json.


## Data Cleaning

In [5]:
df = pd.read_json(clean_file)

# Get the first row of the DataFrame
first_row = df.iloc[0]

# Create a dictionary mapping the old column names to the new ones
column_mapping = {f'Response {i+1}': first_row[f'Response {i+1}'] for i in range(5)}

# Rename the columns
df.rename(columns=column_mapping, inplace=True)

# Drop the first row
df.drop(df.index[0], inplace=True)

# Generate a list of 'Company' columns
company_cols = [col for col in df.columns if 'Company' in col]

# Convert the numeric 'Company' values to float and round them
for col in company_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].round(1)


# Fill NaN values with 0
df.fillna(0, inplace=True)

# Print the DataFrame
print(df.head(20))
print(df.info())

   Project                                Question level 1   Question level 2 Question level 3  Company A  Company B  Company C  Company D  Company E
1    Alpha    What was your total AuM in the last 5 years?           EoY 2020            Total        0.0     3550.0     6000.0     6857.0        0.0
2    Alpha    What was your total AuM in the last 5 years?           EoY 2021            Total        0.0     3710.0     6211.0     6843.0     5138.0
3    Alpha    What was your total AuM in the last 5 years?           EoY 2022            Total        0.0     3940.0     9358.0     6737.0     5698.0
4    Alpha    What was your total AuM in the last 5 years?           EoY 2023            Total        0.0     4060.0     6521.0     6827.0     5745.0
5    Alpha    What was your total AuM in the last 5 years?           EoY 2024            Total        0.0     4251.0     6755.0     7201.0     5717.0
6    Alpha    What was your total AuM in the last 5 years?  YTD 31 March 2024            Total      

In [6]:
# drop EoY 2024 in 'Question level 2' as EoY 2024 is not a valid value in the real world scenario
df = df[df['Question level 2'] != 'EoY 2024']


# Calculate growth for each 'Response' column
for col in company_cols:
    growth_col = col + ' Growth'
    df[growth_col] = df[col].pct_change() * 100  # Growth percentage

print(df.head(20))
print(df.info())

   Project                                Question level 1   Question level 2 Question level 3  Company A  Company B  Company C  Company D  Company E  Company A Growth  Company B Growth  Company C Growth  Company D Growth  Company E Growth
1    Alpha    What was your total AuM in the last 5 years?           EoY 2020            Total        0.0     3550.0     6000.0     6857.0        0.0               NaN               NaN               NaN               NaN               NaN
2    Alpha    What was your total AuM in the last 5 years?           EoY 2021            Total        0.0     3710.0     6211.0     6843.0     5138.0               NaN          4.507042          3.516667         -0.204171               inf
3    Alpha    What was your total AuM in the last 5 years?           EoY 2022            Total        0.0     3940.0     9358.0     6737.0     5698.0               NaN          6.199461         50.668169         -1.549028         10.899183
4    Alpha    What was your total AuM in

In [7]:
# Transpose the DataFrame
df = df.transpose()

# Print the transposed DataFrame
print(df)


                                                            1                                             2                                             3                                             4                                             6                                               7                                               8                                               9                                               10                                              11  ...                                              44                                              45                                              46                                              47                                              48                                          49                                          50                                          51                                          52                                          53
Project                                         

## Calculations

In [8]:
# # AUM 4 year CAGR
# df['AUM 4 year CAGR'] = np.nan
# df['AUM 4 year CAGR'] = df['AUM 4 year CAGR'].astype(float)
# df['AUM 4 year CAGR'] = (df['AUM 2024'] / df['AUM 2020']) ** (1 / 4) - 1

print(df.columns)

Index([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 42, 43, 44,
       45, 46, 47, 48, 49, 50, 51, 52, 53],
      dtype='int64')


## Summary